**Cabella's** --> *interactive, zoom + click to view small boxes*


RUN THE CELLS IN ORDER, ONLY 1st WORD OF BRAND NAME DISPLAYED

**Big Box Label**: Product Category, 
**Sub Box Label**: Brand,
**Box Color**: Avg Review, 
**Box Size**: # of Reviews

In [ ]:
import pandas as pd 
import numpy as np 
import requests 
import time 
import bs4
from bs4 import BeautifulSoup
import random
import re
import pandas as pd
import re
import numpy as np
import pandas as pd
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

In [ ]:
urlc = 'https://www.cabelas.com/shop/en/mens-clothing'
cookies = {'vapidPublicKey': 'BGyZkb_FW0C0d1K6sY3qfwQGUFpXMWjqNYFBBBGwyW3yTEq0iwkcMJnfmNcYN6KyxfgSQ0O37FZ6FMyysLJ-Rus=','appKey': 'GaQQtg6eREiAeioiqriWqw','token': 'MTpHYVFRdGc2ZVJFaUFlaW9pcXJpV3F3OnpXX3pyM242WXhKVWFQUVlCQmhSaFk5RjllS0dMemM3RnRNdDVGa3Zmclk'}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
html = requests.get(urlc, headers = headers).text
soupce = bs4.BeautifulSoup(html, "html.parser")

catLink = []
cat_link = []
prod_links = []
cat = []
prod_name = []
prod_price = []
prod_rating = []
prod_numRating = []
for xi in range(len(str(soupce).split('<a class="menuLink" href='))-1):
  xi = xi+1
  cat_link.append(str(soupce).split('<a class="menuLink" href=')[xi].split('"')[1])



In [ ]:
for xo in range(len(cat_link)):
  time.sleep(random.uniform(0,6))
  urlc = cat_link[xo]
  html = requests.get(urlc, headers = headers).text
  soupc = bs4.BeautifulSoup(html, "html.parser")

  for xu in range(len(str(soupc).split('<a ;="" aria-hidden="true" href='))-1):
    try:
      xu = xu+1
      prod_links.append(str(soupc).split('<a ;="" aria-hidden="true" href=')[xu].split('"')[1])
      prod_price.append(str(soupc).split('<a ;="" aria-hidden="true" href=')[xu].split('<input id="analyticsWishlist_unitPrice_" name="unitPrice" type="hidden" value=')[1].split('"')[1])
      cat.append(cat_link[xo].split('https://www.cabelas.com/shop/en/')[1])
      catLink.append(cat_link[xo])
    except:
      prod_links.append(0)
      prod_price.append(0)
      cat.append(cat_link[xo].split('https://www.cabelas.com/shop/en/')[1])
      catLink.append(cat_link[xo])


    revString = str(soupc).split('<a ;="" aria-hidden="true" href=')[xu].split('"></span> <span class="bvReviews"')[0].split('<span class=')
    cv = 0
    try:
      prod_name.append(str(soupc).split('<a ;="" aria-hidden="true" href=')[xu].split('>')[1].split('<')[0])
      prod_numRating.append(str(soupc).split('<a ;="" aria-hidden="true" href=')[xu].split('<span class="bvReviews"> (')[1].split(')')[0])
    except:
      prod_name.append(0)
      prod_numRating.append(0)
    for xm in range(5):
      xm=xm+1
      try:
        revString[xm].split('bpsRatingStarFull')[1]
        cv = cv+1
      except:
        if len(revString[xm].split('RatingStar'))==3:
          num = .01*int(revString[xm].split('"bpsRatingStarHalf bpsRatingStarHalf')[1].split('"')[0])
          cv = cv+num
        else:
          try:
            num = .01*int(revString[xm].split('"bpsRatingStarHalf')[1].split('"')[0])
            cv = cv+num
          except:
            if revString[xm].split('"bpsRatingStarEmpty')[1].split('"')[0] == '':
              num = 0
              cv = cv
            else:
              num = .01*int(revString[xm].split('"bpsRatingStarEmpty')[1].split('"')[0])
              cv = cv+num
    try:
      prod_rating.append(cv)
    except:
      prod_rating.append(0)

In [ ]:
prodRat= [float(i) for i in prod_rating]
prodRatNum= [int(i) for i in prod_numRating]
prodPrat = [float(i) for i in prod_price]
df = pd.DataFrame()
df['cat'] = cat[0:len(cat)]
df['links'] = prod_links[0:len(cat)]
df['prod_namess'] = prod_name[0:len(cat)]
df['ratings'] = prodRat[0:len(cat)]
df['rating_count'] = prodRatNum[0:len(cat)]
df['price'] = prodPrat[0:len(cat)]
df = df[df['cat']!='home-gifts-new-arrivals']
df = df[df['cat']!='camping-new']
df = df[df['cat']!='drinkware']
df = df.replace(0,1)
df.to_csv('final_luth.csv')
cabelas = df
short_name = []
for xi in range(len(cabelas['prod_namess'])):
  try:
    short_name.append(cabelas['prod_namess'][xi].split()[0].replace(',',''))
  except:
    short_name.append(0)
cabelas['short_name'] = short_name

import plotly.express as px 
fig3 = px.treemap(cabelas, path=['cat','short_name'], values='rating_count', color='ratings')
fig3.show()